# Predicting house prices from the Ames dataset

As usual, we'll follow our general machine learning workflow:
1. Define the problem.
2. Get training and test data.
3. Wrangle, prepare, and clean the data.
4. Analyze, identify patterns, and explore the data.
5. Model, predict, solve the problem.
6. Visualize, report, and present the steps and solution.
7. Publish the results.

## Define the problem

Kaggle makes this part easy for us.

**Problem:** Given a set of features about homes in Ames, Iowa, predict their final prices.

## Fetch the data

In [1144]:
import pandas as pd

# Data
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')
full = [train, test]

# Helper functions
# Given a feature, pivot training data against SalePrice
def corr_with_price(feature):
    return train[[feature, 'SalePrice']].groupby([feature], as_index=False).mean().sort_values(by='SalePrice', ascending=False)

def corr_with_price_and_count(feature):
    return train[[feature, 'SalePrice']].groupby([feature]).agg(['mean', 'count'])

## Explore and wrangle the data

Steps three and four in the general machine learning research workflow should really be combined, there's so much hopping between them.

What features do we got?

In [1145]:
print(train.columns.values)

['Id' 'MSSubClass' 'MSZoning' 'LotFrontage' 'LotArea' 'Street' 'Alley'
 'LotShape' 'LandContour' 'Utilities' 'LotConfig' 'LandSlope'
 'Neighborhood' 'Condition1' 'Condition2' 'BldgType' 'HouseStyle'
 'OverallQual' 'OverallCond' 'YearBuilt' 'YearRemodAdd' 'RoofStyle'
 'RoofMatl' 'Exterior1st' 'Exterior2nd' 'MasVnrType' 'MasVnrArea'
 'ExterQual' 'ExterCond' 'Foundation' 'BsmtQual' 'BsmtCond' 'BsmtExposure'
 'BsmtFinType1' 'BsmtFinSF1' 'BsmtFinType2' 'BsmtFinSF2' 'BsmtUnfSF'
 'TotalBsmtSF' 'Heating' 'HeatingQC' 'CentralAir' 'Electrical' '1stFlrSF'
 '2ndFlrSF' 'LowQualFinSF' 'GrLivArea' 'BsmtFullBath' 'BsmtHalfBath'
 'FullBath' 'HalfBath' 'BedroomAbvGr' 'KitchenAbvGr' 'KitchenQual'
 'TotRmsAbvGrd' 'Functional' 'Fireplaces' 'FireplaceQu' 'GarageType'
 'GarageYrBlt' 'GarageFinish' 'GarageCars' 'GarageArea' 'GarageQual'
 'GarageCond' 'PavedDrive' 'WoodDeckSF' 'OpenPorchSF' 'EnclosedPorch'
 '3SsnPorch' 'ScreenPorch' 'PoolArea' 'PoolQC' 'Fence' 'MiscFeature'
 'MiscVal' 'MoSold' 'YrSold' 'SaleTy

Beautiful, and we've got more details in the data description text file. Let's take some high-level glances at our dataset, and then go through each feature and dig a little deeper.

In [1146]:
train.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [1147]:
train.info()
print('_'*40)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

We've got 1460 rows of training data and 1459 rows of test data. Quite a few columns in each set with missing values.

**MSSubClass:** Identifies the type of dwelling involved in the sale.
* 20: 1-story 1946 and newer all styles
* 30: 1-story 1945 and older
* 40: 1-story w/finished attic all ages
* 45: 1-1/2 story - unfinished all ages
* 50: 1-1/2 story finished all ages
* 60: 2-story 1946 and newer
* 70: 2-story 1945 and older
* 75: 2-1/2 story all ages
* 80: split or multi-level
* 85: split foyer
* 90: duplex - all styles and ages
* 120: 1-story pud (planned unit development) - 1946 and newer
* 150: 1-1/2 story pud - all ages
* 160: 2-story pud - 1946 and newer
* 180: pud - multilevel - incl split lev/foyer
* 190: 2 family conversion - all styles and ages

In [1148]:
corr_with_price('MSSubClass')

,MSSubClass,SalePrice
5,60,239948
11,120,200779
7,75,192437
0,20,185224
8,80,169736
6,70,166772
2,40,156125
9,85,147810
4,50,143302
12,160,138647


In [1149]:
ms_sub_class_dummies_train = pd.get_dummies(data=train['MSSubClass'], prefix='MSSubClass_')
train = pd.concat([train, ms_sub_class_dummies_train], axis=1)

ms_sub_class_dummies_test = pd.get_dummies(data=test['MSSubClass'], prefix='MSSubClass_')
test = pd.concat([test, ms_sub_class_dummies_test], axis=1)

In [1150]:
train.drop(['MSSubClass'], axis=1, inplace=True)
test.drop(['MSSubClass'], axis=1, inplace=True)

full = [train, test]

Now we've turned that mess of a column into a set of one-hot columns in each dataset and dropped the original. A few more of those and we'll be ready for some regression.

What's next, MSZoning?

In [1151]:
mszoning_dummies_train = pd.get_dummies(data=train['MSZoning'], prefix='MSZoning_')
train = pd.concat([train, mszoning_dummies_train], axis=1)
mszoning_dummies_test = pd.get_dummies(data=test['MSZoning'], prefix='MSZoning_')
test = pd.concat([test, mszoning_dummies_test], axis=1)

train.drop(['MSZoning'], axis=1, inplace=True)
test.drop(['MSZoning'], axis=1, inplace=True)

full = [train, test]

The above table gives us the mean SalePrice for each MSSubClass (house type).

The most expensive kind of house seems to be the 2-story 1946 and newer (MSSubClass=60), with the cheapest being 1-story 1945 and older (MSSubClass=30). Might be worth engineering a feature for Old vs. New houses, or Unfinished vs. Finished, or PUD vs. non-PUD, or 1 vs. 1-1/2 vs. 2 story. But all in all it seems like a useful feature, so let's keep it and move on.

Next feature, **MSZoning**: Identifies the general zoning classification of the sale.
* A: Agriculture
* C: Commercial
* FV: Floating Village Residential
* I: Industrial
* RH: Residential High Density
* RL: Residential Low Density
* RP: Residential Low Density Park
* RM: Residential Medium Density

In [1152]:
train['MSZoning'].describe()

KeyError: 'MSZoning'

In [ ]:
glance_at_correlation('MSZoning')

In [ ]:
train.groupby(['MSZoning']).size()

Most homes are Residential Low Density. Few homes outside of this, but highest value at Floating Village Residential. Might still be worth something.

Next: **LotFrontage**: Linear feet of street connected to property.

In [ ]:
train['LotFrontage'].describe()

The first thing to notice here is that we only have 1201 out of 1460 non-null values. We're gonna need a way to fill those values in, in order to effectively explore this data.

Finding the median LotFrontage value given some other set of features would be the ideal way to fill in this column. We'll log that in our to-do list and come back to it once we've checked out the rest of our features.

**LotArea:** Lot size in square feet.

In [ ]:
train['LotArea'].describe()

Full up! Since it's a float, a raw pivot table would be hard to make sense of. Let's group these values into bands and pivot on that!

In [ ]:
train['LotAreaBand'] = pd.cut(train['LotArea'], 4)
glance_at_correlation_and_count('LotAreaBand')

What a horribly uneven distribution! Virtually all houses in the dataset have a low LotArea, between 1086 and 54786 units. It does seem like it might be correlated with the SalePrice though, so let's keep it and do further reengineering later.

Whadup with **Street**?

*Type of road access to property*
* Grvl: Gravel
* Pave: Paved

We can easily flip those values to 0s and 1s, but first let's make sure they matter!

In [ ]:
train.groupby(['Street']).size()

Virtually all of our training houses have paved street access, making this likely a useless feature. Just to be sure, let's pivot the column anyway.

In [ ]:
glance_at_correlation('Street')

Well, it actually seems like it might not be totally useless. Let's keep it and reengineer it later.

#### Alley

*Type of alley access to property*
* Grvl: Gravel
* Pave: Paved
* NA: No alley access

In [ ]:
train.groupby(['Alley']).size()

Our problem here is that NA is a valid data value in this column, but they're all registered as NaNs. Let's convert this column into 0, 1, and 2 values.

In [ ]:
train['Alley'].fillna(0, inplace=True)
test['Alley'].fillna(0, inplace=True)
for dataset in full:
    dataset['Alley'].fillna(0, inplace=True)
    dataset.loc[dataset['Alley'] == 'Grvl', 'Alley'] = 1
    dataset.loc[dataset['Alley'] == 'Pave', 'Alley'] = 2
    
full = [train, test]
train.head()

In [ ]:
train.groupby(['Alley']).size()

In [ ]:
glance_at_correlation('Alley')

No Alley commands the highest price (though has the most wiggle-room due to overwhelming representation), followed by Paved, with Gravel bringing up the rear by quite a margin. Hopefully our models can make use of this.

#### LotShape

*General shape of property*
* Reg: Regular
* IR1: Slightly irregular
* IR2: Moderately irregular
* IR3: Irregular

In [ ]:
train.groupby(['LotShape']).size()

In [ ]:
train['LotShape'].describe()

In [ ]:
glance_at_correlation_and_count('LotShape')

Irregular properties look like they command higher prices than regular ones! We'll certainly keep this feature, but we may need to reengineer it later (regular vs. irregular?).

**LandContour**

*Flatness of the property*
* Lvl: Near Flat/Level
* Bnk: Banked - Quick and significant rise from street grade to building
* HLS: Hillside - Significant slope from side to side
* Low: Depression

In [ ]:
train.groupby(['LandContour']).size()

In [ ]:
train['LandContour'].describe()

In [ ]:
glance_at_correlation('LandContour')

Oh these definitely look correlated. I mean I'm not a doctor or anything, but let's keep the LandContour feature. We are going to need to make it categorical, but we'll do that later.

**Utilities**

*Type of utilities available*
* AllPub: All public Utilities (E,G,W,&S)
* NoSewr: Electricity, Gas, and Water (Septic Tank)
* NoSeWa: Electricity and Gas Only
* ELO: Electricity only

In [ ]:
train['Utilities'].describe()

So it's entirely AllPub except for one. Let's take a closer look at that one.

In [ ]:
glance_at_correlation_and_count('Utilities')

Using my common-sense, homes without access to all public utilities will command a lower final SalePrice than homes with all public utilities. Gonna keep it.

**LotConfig**: Lot configuration
* Inside: Inside lot
* Corner: Corner lot
* CulDSac: Cul-de-sac
* FR2: Frontage on 2 sides of property
* FR3: Frontage on 3 sides of property

In [ ]:
train['LotConfig'].describe()

In [ ]:
glance_at_correlation('LotConfig')

Awesome. I feel like I must not be high enough with my standards here, because I seem hellbent on keeping and reengineering most of these features.

**LandSlope**: Slope of property
* Gtl: Gentle slope
* Mod: Moderate slope
* Sev: Severe slope

My guess? Gentle slopes are correlated with higher SalePrice than severe slopes. But what's our data say?

In [ ]:
train['LandSlope'].describe()

In [ ]:
glance_at_correlation('LandSlope')

Ooh I was so very wrong! I suppose a more severe slope may indicate living on a hill, which is where all the fancy people live (so I've been told). Definitely a keeper. This next feature should be interesting...

**Neighborhood**: Physical locations within Ames city limits
* Blmngtn: Bloomington Heights
* Blueste: Bluestem
* BrDale: Briardale
* BrkSide: Brookside
* ClearCr: Clear Creek
* CollgCr: College Creek
* Crawfor: Crawford
* Edwards: Edwards
* Gilbert: Gilbert
* IDOTRR: Iowa DOT and Rail Road
* MeadowV: Meadow Village
* Mitchel: Mitchell
* Names: North Ames
* NoRidge: Northridge
* NPkVill: Northpark Villa
* NridgeHt: Northridge Heights
* NWAmes: Northwest Ames
* OldTown: Old Town
* SWISU: South & West of Iowa State University
* Sawyer: Sawyer
* SaywerW: Sawyer West
* Somerst: Somerset
* StoneBr: Stone Brook
* Timber: Timberland
* Veenker: Veenker

My guess is that this is a really great feature with a little reengineering. What neighborhoods are correlated with high house SalePrices? Low ones?

In [ ]:
train['Neighborhood'].describe()

In [ ]:
glance_at_correlation('Neighborhood')

It's times like these when I wonder which approach wouldn't create a better model: combining some Neighborhoods into groups, so we have four or five NeighborhoodBands tiered by SalePrice (i.e., MeadowV and IDOTRR and BrDale together, NridgeHt and NoRidge together, etc...), or feeding the model the raw Neighborhood breakdown like so? I'm inclined to leave them separate, like they are, and just cast the column to category.

**Condition1**: Proximity to various conditions
* Artery: adjacent to arterial street
* Feedr: adjacent to feeder street
* Norm: normal
* RRNn: within 200' of North-South Railroad
* RRAn: adjacent to North-South Railroad
* PosN: near positive off-site feature--park, greenbelt, etc.
* PosA: adjacent to positive off-site feature
* RRNe: within 200' of East-West Railroad
* RRAe: adjacent to East-West Railroad

In [ ]:
train['Condition1'].describe()

In [ ]:
glance_at_correlation('Condition1')

In [ ]:
train['Condition2'].describe()

In [ ]:
glance_at_correlation('Condition2')

Condition1 and Condition2 together certainly make for powerful predictive features. Trying to think of how we can reengineer a more effective combined feature...

**BldgType**: Type of dwelling
* 1Fam: single-family detached
* 2FmCon: two-family conversion; originally built as one-family dwelling
* Duplx: duplex
* TwnhsE: townhouse end unit
* TwnhsI: townhouse inside unit

In [ ]:
train['BldgType'].describe()

In [ ]:
glance_at_correlation('BldgType')

In [ ]:
glance_at_correlation_and_count('BldgType')

I'm clearly going to need to apply more sophisticated methods of comparing features. Determine statistical significance of the mean given the count and relative count, chart these differently (and against each other in terms of impact rather than relative distribution)...

Anyway, onto the next one:

**HouseStyle**: Style of dwelling
* 1Story: one story
* 1.5Fin: one and one-half story: 2nd level finished
* 1.5Unf: one and one-half story: 2nd level unfinished
* 2Story: two story
* 2.5Fin: two and one-half story: 2nd level finished
* 2.5Unf: two and one-half story: 2nd level unfinished
* SFoyer: split foyer
* SLvl: split level

In [ ]:
train['HouseStyle'].describe()

In [ ]:
glance_at_correlation('HouseStyle')

In [ ]:
train['HouseStyle'].value_counts()

In [ ]:
train.corr()['LotFrontage']['SalePrice']

In [ ]:
train.corr()['LotArea']['SalePrice']

In [ ]:
train.corr()['YearBuilt']['SalePrice']

In [ ]:
train.corr()['YearRemodAdd']['SalePrice']

In [ ]:
train.corr()['MasVnrArea']['SalePrice']

In [ ]:
train.corr()['BsmtFinSF1']['SalePrice']

In [ ]:
train.corr()['BsmtFinSF2']['SalePrice']

In [ ]:
train.corr()['BsmtUnfSF']['SalePrice']

In [ ]:
train.corr()['TotalBsmtSF']['SalePrice']

In [ ]:
train.corr()['1stFlrSF']['SalePrice']

In [ ]:
train.corr()['2ndFlrSF']['SalePrice']

In [ ]:
train.corr()['LowQualFinSF']['SalePrice']

In [ ]:
train.corr()['GrLivArea']['SalePrice']

In [ ]:
train.corr()['BsmtHalfBath']['SalePrice']

In [ ]:
train.corr()['BsmtFullBath']['SalePrice']

In [ ]:
train.corr()['FullBath']['SalePrice']

In [ ]:
train.corr()['HalfBath']['SalePrice']

In [ ]:
train.corr()['BedroomAbvGr']['SalePrice']

In [ ]:
train.corr()['KitchenAbvGr']['SalePrice']

In [ ]:
train.corr()['TotRmsAbvGrd']['SalePrice']

In [ ]:
train.corr()['Fireplaces']['SalePrice']

In [ ]:
train.corr()['GarageYrBlt']['SalePrice']

In [ ]:
train.corr()['GarageCars']['SalePrice']

In [ ]:
train.corr()['GarageArea']['SalePrice']

In [ ]:
train.corr()['WoodDeckSF']['SalePrice']

In [ ]:
train.corr()['OpenPorchSF']['SalePrice']

In [ ]:
train.corr()['EnclosedPorch']['SalePrice']

In [ ]:
train.corr()['3SsnPorch']['SalePrice']

In [ ]:
train.corr()['ScreenPorch']['SalePrice']

In [ ]:
train.corr()['PoolArea']['SalePrice']

In [ ]:
train.corr()['MiscVal']['SalePrice']

In [ ]:
train.corr()['YrSold']['SalePrice']

In [ ]:
train.corr()['MoSold']['SalePrice']

#### Deep down the rabbit hole

I imagine I need to combine similar features, turn everything into numbers, and run some regressions.

I bet I come back to this notebook in a couple years and slap myself for being so foolish, but let's enjoy it while it lasts, yeah?

In [ ]:
train['YearBuilt'].min()

In [ ]:
for dataset in full:
    dataset['SellingAge'] = dataset['YrSold'] - dataset['YearBuilt']

train['SellingAgeBand'] = pd.cut(train['SellingAge'], 6)
glance_at_correlation_and_count('SellingAgeBand')

The newest, followed by the oldest. Not sure how to reengineer this one. Reducing the bands eliminates the historic age bump, but including it confuses the trend. I wonder what the effect would be if I turned it into three uneven bands: New (< 22.667), Old (22.667 >= x >= 113.333), and Ancient (> 113.333).

In [ ]:
for dataset in full:
    dataset.loc[dataset['SellingAge'] <= 22.667, 'SellingAge'] = 0
    dataset.loc[dataset['SellingAge'] > 113.333, 'SellingAge'] = 1
    dataset.loc[(dataset['SellingAge'] > 22.667) & (dataset['SellingAge'] <= 113.333), 'SellingAge'] = 2
    
train = train.drop(['SellingAgeBand'], axis=1)
full = [train, test]

glance_at_correlation('SellingAge')

I'm gonna go out on a limb here and venture to guess that the month the house was sold has little to no bearing on the final SalePrice. Let's take a closer look just in case... Maybe we'll turn them into a new feature, SeasonOfSale.

In [ ]:
train['MoSold'] = train['MoSold'].astype(int)

train.loc[train['MoSold'].isin([9,10,11]), 'SeasonOfSale'] = 0
train.loc[train['MoSold'].isin([12,1,2]), 'SeasonOfSale'] = 1
train.loc[train['MoSold'].isin([3,4,5]), 'SeasonOfSale'] = 2
train.loc[train['MoSold'].isin([6,7,8]), 'SeasonOfSale'] = 3

In [ ]:
glance_at_correlation_and_count('SeasonOfSale')

Yeah zero correlation, kill it with fire.

In [ ]:
for dataset in full:
    dataset = dataset.drop(['MoSold'], axis=1)
    
train = train.drop(['SeasonOfSale'], axis=1)

full = [train, test]

In [ ]:
glance_at_correlation_and_count('SaleType')

In [ ]:
train.loc[train['SaleType'] == 'CWD']